In [2]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

In [3]:
link='https://en.wikipedia.org/wiki/List_of_American_films_of_2020'

In [4]:
source=urllib.request.urlopen(link).read()
soup=bs.BeautifulSoup(source,'lxml')

In [6]:
tables=soup.find_all('table',class_='wikitable sortable')

In [ ]:
len(tables)

In [7]:
type(tables[0])

bs4.element.Tag

In [8]:
df1=pd.read_html(str(tables[0]))[0]
df2=pd.read_html(str(tables[1]))[0]
df3=pd.read_html(str(tables[2]))[0]
df4=pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [9]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

C:\Users\MAHAK\AppData\Local\Temp\ipykernel_18824\1752420719.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)


In [10]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,J A N U A R Y,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2],NaN
1,J A N U A R Y,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3],NaN
2,J A N U A R Y,10,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4],NaN
3,J A N U A R Y,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN,NaN
4,J A N U A R Y,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5],NaN
...,...,...,...,...,...,...,...
270,D E C E M B E R,25,We Can Be Heroes,Netflix / Troublemaker Studios / Double R Prod...,Robert Rodriguez (director/screenplay); Priyan...,NaN,[241]
271,D E C E M B E R,25,News of the World,Universal Pictures / Perfect World Pictures,Paul Greengrass (director/screenplay); Luke Da...,NaN,[242]
272,D E C E M B E R,25,One Night in Miami...,Amazon Studios,Regina King (director); Kemp Powers (screenpla...,NaN,[243]
273,D E C E M B E R,25,Promising Young Woman,Focus Features / FilmNation Entertainment,Emerald Fennell (director/screenplay); Carey M...,NaN,[244]


In [11]:
df_5 = df[['Title','Cast and crew']]

In [12]:
df_5

,Title,Cast and crew
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...
1,Underwater,"William Eubank (director); Brian Duffield, Ada..."
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col..."
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...
...,...,...
270,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...
271,News of the World,Paul Greengrass (director/screenplay); Luke Da...
272,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...
273,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...


In [13]:
#pip install tmdbv3api
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '8adad600aa361bb1349465ffecb76997'

In [17]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [19]:
df_5['genres'] = df_5['Title'].map(lambda x: get_genre(str(x)))

C:\Users\MAHAK\AppData\Local\Temp\ipykernel_18824\4160290683.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5['genres'] = df_5['Title'].map(lambda x: get_genre(str(x)))


In [21]:
def director(x):
    if "(director)" in x:
        return x.split("(director)")[0]
    elif "(directors)" in x:
        return x.split("(directors)")[0]
    else:
        return x.split("(director/screenplay)")[0]

In [22]:
df_5['director_name'] = df_5['Cast and crew'].map(lambda x: director(str(x)))

C:\Users\MAHAK\AppData\Local\Temp\ipykernel_18824\2399458305.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5['director_name'] = df_5['Cast and crew'].map(lambda x: director(str(x)))


In [23]:
def actor_1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [25]:
df_5['actor_1_name'] = df_5['Cast and crew'].map(lambda x: actor_1(x))

C:\Users\MAHAK\AppData\Local\Temp\ipykernel_18824\1375227415.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5['actor_1_name'] = df_5['Cast and crew'].map(lambda x: actor_1(x))


In [26]:
def actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [27]:
df_5['actor_2_name'] = df_5['Cast and crew'].map(lambda x: actor2(x))

C:\Users\MAHAK\AppData\Local\Temp\ipykernel_18824\2254440078.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5['actor_2_name'] = df_5['Cast and crew'].map(lambda x: actor2(x))


In [28]:
def actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [29]:
df_5['actor_3_name'] = df_5['Cast and crew'].map(lambda x: actor3(x))

In [30]:
df_5= df_5.rename(columns={'Title':'movie_title'})

In [31]:
data_5 = df_5.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [32]:
data_5['actor_2_name'] = data_5['actor_2_name'].replace(np.nan, 'unknown')
data_5['actor_3_name'] = data_5['actor_3_name'].replace(np.nan, 'unknown')

In [33]:
data_5['movie_title']=data_5['movie_title'].str.lower()

In [35]:
data_5['comb']=data_5['actor_1_name'] + ' ' + data_5['actor_2_name'] + ' ' + data_5['actor_3_name'] + ' ' + data_5['director_name'] + " " + data_5['genres']  

In [36]:
data_5.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           3
movie_title      0
comb             3
dtype: int64

In [37]:
data_5 = data_5.dropna(how='any')

In [42]:
data_5.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [38]:
data_5

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller,the grudge,Andrea Riseborough Demián Bichir John Cho Nico...
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction,underwater,Kristen Stewart Vincent Cassel Jessica Henwick...
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,like a boss,Tiffany Haddish Rose Byrne Salma Hayek Miguel ...
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,three christs,Richard Gere Peter Dinklage Walton Goggins Jon...
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,inherit the viper,Josh Hartnett Margarita Levieva Chandler Riggs...
...,...,...,...,...,...,...,...
270,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Action Fantasy Family Comedy,we can be heroes,Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...
271,Paul Greengrass,Tom Hanks,Helena Zengel,unknown,Drama Western Adventure Action,news of the world,Tom Hanks Helena Zengel unknown Paul Greengras...
272,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,one night in miami...,Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...
273,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,promising young woman,Carey Mulligan Bo Burnham Alison Brie Emerald ...


In [39]:
data_upto4=pd.read_csv('data3&4.csv')

In [40]:
# final data or data_upto 5
data_upto5=data_upto4.append(data_5,ignore_index=True)

C:\Users\MAHAK\AppData\Local\Temp\ipykernel_18824\3854006697.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_upto5=data_upto4.append(data_5,ignore_index=True)


In [41]:
data_upto5.to_csv('data5.csv')